### PromptTemplate 
* [PromptTemplate](https://python.langchain.com/api_reference/core/prompts/langchain_core.prompts.prompt.PromptTemplate.html#langchain_core.prompts.prompt.PromptTemplate)
* [ChatPromptTemplate](https://python.langchain.com/api_reference/core/prompts/langchain_core.prompts.chat.ChatPromptTemplate.html#langchain_core.prompts.chat.ChatPromptTemplate)
* [ChatMessagePromptTemplate](https://python.langchain.com/api_reference/core/prompts/langchain_core.prompts.chat.ChatMessagePromptTemplate.html#langchain_core.prompts.chat.ChatMessagePromptTemplate)
* [FewShotPromptTemplate](https://python.langchain.com/api_reference/core/prompts/langchain_core.prompts.few_shot.FewShotPromptTemplate.html#langchain_core.prompts.few_shot.FewShotPromptTemplate)
* PartialPrompt

In [3]:
# poetry add python-dotenv langchain langchain-openai

In [2]:
from dotenv import load_dotenv
import os
# .env 파일을 불러와서 환경 변수로 설정
load_dotenv(dotenv_path='../.env')

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
print(OPENAI_API_KEY[:2])

gs


##### 1) PromptTemplate 의 from_template() 함수 사용
* 주로 LLM(텍스트 완성형 모델, ex. Ollama, GPT-3.5)과 함께 사용
* 하나의 문자열 프롬프트를 생성

In [5]:
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from pprint import pprint

template_text = "{model_name} 모델의 학습 원리를 {count} 문장으로 한국어로 답변해 주세요."

# PromptTemplate 인스턴스를 생성
prompt_template = PromptTemplate.from_template(template_text)

# llm = ChatOpenAI(model="gpt-3.5-turbo-0125")
llm = ChatOpenAI(
    api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    # model="meta-llama/llama-4-scout-17b-16e-instruct",  # Spring AI와 동일한 모델
    model = "openai/gpt-oss-120b",                # GPT-OSS-120B
    temperature=0.7
)

chain = prompt_template | llm | StrOutputParser()
response = chain.invoke({"model_name":"ChatGPT", "count":3})
pprint(response)

('ChatGPT는 대규모 텍스트 데이터를 사용해 다음에 올 단어를 예측하도록 하는 **자기지도 학습**으로 사전 학습됩니다. 이후 인간이 '
 '만든 질문‑답변 쌍 등으로 구성된 **지도 학습**과, 모델의 출력을 인간 피드백으로 평가해 보강하는 **강화 학습**(RLHF)을 거쳐 '
 '사용자의 의도에 맞는 답변을 생성하도록 미세 조정됩니다. 이렇게 단계별로 학습된 파라미터들은 입력된 문맥을 이해하고, 그에 맞는 '
 '자연스러운 언어 출력을 생성하는 기반이 됩니다.')


##### 2) PromptTemplate 결합하기
* 동일한 Prompt 패턴을 사용하지만 여러 개의 질문을 작성해서 LLM을 실행할 수도 있습니다.

In [7]:
template_text = "{model_name} 모델의 학습 원리를 {count} 문장으로 한국어로 답변해 주세요."

# PromptTemplate 인스턴스를 생성
prompt_template = PromptTemplate.from_template(template_text)

# 템플릿에 format()을 써서 값을 채워서 프롬프트를 완성
filled_prompt = prompt_template.format(model_name="ChatGPT", count=3)

# 문자열 템플릿 결합 (PromptTemplate + PromptTemplate + 문자열)
combined_prompt = (
              prompt_template
              + PromptTemplate.from_template("\n\n 그리고 {model_name} 모델의 장점을 요약 정리해 주세요")
              + "\n\n {model_name} 모델과 비슷한 AI 모델은 어떤 것이 있나요? 모델명은 {language}로 답변해 주세요."
)
combined_prompt.format(model_name="ChatGPT", count=3, language="영어")
print(combined_prompt)

#llm = ChatOpenAI(model="gpt-3.5-turbo-0125")
chain = combined_prompt | llm | StrOutputParser()
response = chain.invoke({"model_name":"ChatGPT", "count":3, "language":"영어"})

pprint(response)

input_variables=['count', 'language', 'model_name'] input_types={} partial_variables={} template='{model_name} 모델의 학습 원리를 {count} 문장으로 한국어로 답변해 주세요.\n\n 그리고 {model_name} 모델의 장점을 요약 정리해 주세요\n\n {model_name} 모델과 비슷한 AI 모델은 어떤 것이 있나요? 모델명은 {language}로 답변해 주세요.'
('**ChatGPT 모델의 학습 원리 (3문장)**  \n'
 '1. 대규모 텍스트 데이터를 이용해 트랜스포머 기반의 언어 모델을 사전 학습(pre‑training)하고, 문맥을 예측하는 마스크드 언어 '
 '모델링과 다음 문장 예측 같은 목표를 최적화합니다.  \n'
 '2. 사전 학습된 모델을 실제 대화나 특정 작업에 맞게 인간 피드백을 활용한 강화학습(RLHF) 단계에서 미세 '
 '조정(fine‑tuning)하여 응답 품질과 안전성을 높입니다.  \n'
 '3. 학습 과정에서는 역전파와 옵티마이저(예: Adam)로 파라미터를 업데이트하며, 수십억 개의 파라미터가 서로 연결된 네트워크 형태로 '
 '지식을 내재화합니다.  \n'
 '\n'
 '---\n'
 '\n'
 '### ChatGPT 모델의 장점 요약  \n'
 '\n'
 '- **다양한 언어와 도메인에 대한 높은 이해도**: 방대한 텍스트 코퍼스를 학습해 일상 대화, 전문 지식, 코딩 등 다양한 분야에서 '
 '자연스러운 응답을 생성합니다.  \n'
 '- **문맥 유지 및 연속적인 대화**: 트랜스포머 구조가 긴 문맥을 효과적으로 처리해 이전 발언을 기억하고 일관된 흐름을 '
 '유지합니다.  \n'
 '- **사용자 맞춤형 조정 가능**: RLHF와 파인튜닝을 통해 특정 목적(예: 고객 지원, 교육, 창작) 등에 맞춰 행동을 조정할 수 '
 '있어 유연성이 뛰어납니다.  \n'
 '- **실시간 추론 속도**: 최적화된 모델 아키텍처와 

#### PromptTemplate 의 파라미터를 배열 형태로 하여 여러개 사용하는 경우

In [ ]:
template_text = "{model_name} 모델의 학습 원리를 {count} 문장으로 한국어로 답변해 주세요."

# PromptTemplate 인스턴스를 생성
prompt_template = PromptTemplate.from_template(template_text)

questions = [
    {"model_name": "GPT-4", "count": 2},
    {"model_name": "Gemini", "count": 3},
    {"model_name": "Claude", "count": 4},
]

# 여러 개의 프롬프트를 미리 생성
formatted_prompts = [prompt_template.format(**q) for q in questions]    # ** : 파라미터에 dict가 들어갈때 ** 사용
print(formatted_prompts)  # 미리 생성된 질문 목록 확인

#llm = ChatOpenAI(model="gpt-3.5-turbo-0125")

for prompt in formatted_prompts:
    print(type(prompt), prompt)
    response = llm.invoke(prompt)
    pprint(response.content)

['GPT-4 모델의 학습 원리를 2 문장으로 한국어로 답변해 주세요.', 'Gemini 모델의 학습 원리를 3 문장으로 한국어로 답변해 주세요.', 'Claude 모델의 학습 원리를 4 문장으로 한국어로 답변해 주세요.']
<class 'str'> GPT-4 모델의 학습 원리를 2 문장으로 한국어로 답변해 주세요.
('GPT‑4는 대규모 텍스트 데이터를 이용해 다음에 올 단어를 예측하도록 하는 자기지도 학습(self‑supervised learning) '
 '과정을 거쳐, 언어의 통계적 패턴과 의미 관계를 내부 파라미터에 압축합니다. 학습이 끝난 후에는 인간의 피드백을 활용한 '
 '강화학습(RLHF)으로 답변 품질과 안전성을 추가로 조정합니다.')
<class 'str'> Gemini 모델의 학습 원리를 3 문장으로 한국어로 답변해 주세요.
('Gemini 모델은 대규모 텍스트와 멀티모달 데이터를 사용해 사전 학습(pre‑training) 단계에서 언어와 시각 정보를 동시에 '
 '학습합니다. 이후, 인간 피드백을 활용한 강화학습(RLHF) 과정을 거쳐 원하는 응답을 생성하도록 미세 조정합니다. 이러한 두 단계의 '
 '학습을 결합해 높은 이해력과 생성 능력을 갖춘 모델을 구현합니다.')
<class 'str'> Claude 모델의 학습 원리를 4 문장으로 한국어로 답변해 주세요.
('Claude 모델은 대규모 텍스트 데이터를 사용해 사전 학습(pre‑training)됩니다. 이 과정에서 트랜스포머 기반의 언어 모델이 '
 '문맥을 예측하도록 학습되어, 다음 토큰을 추정하는 확률을 최적화합니다. 이후 인간 피드백을 활용한 강화학습(RLHF) 단계에서, 모델이 '
 '생성한 답변을 사람 평가자가 순위 매기고, 이를 바탕으로 보상 모델을 만들어 정책을 미세 조정합니다. 최종적으로는 다양한 작업에 '
 '적합하도록 여러 도메인에서 추가 미세 튜닝을 수행해 실용성을 높입니다.')


##### 2) ChatPromptTemplate
* Tuple 형태의 system, user, assistant 메시지 지원
* 여러 개의 메시지를 조합하여 LLM에게 전달 가능
* 간결성과 가독성이 높고 단순한 구조

In [9]:
# 2-튜플 형태의 메시지 목록으로 프롬프트 생성 (type, content)

from langchain_core.prompts import ChatPromptTemplate

chat_prompt = ChatPromptTemplate.from_messages([
    # role, message
    ("system", "This system is an expert in answering questions about {topic}. Please provide clear and detailed explanations."),
    ("human", "{model_name} 모델의 학습 원리를 설명해 주세요."),
])

messages = chat_prompt.format_messages(topic="AI", model_name="ChatGPT")
print(messages)

# 생성한 메시지를 바로 주입하여 호출하기
#llm = ChatOpenAI(model="gpt-3.5-turbo-0125")
response = llm.invoke(messages)

print(type(response))
print(response.content)

[SystemMessage(content='This system is an expert in answering questions about AI. Please provide clear and detailed explanations.', additional_kwargs={}, response_metadata={}), HumanMessage(content='ChatGPT 모델의 학습 원리를 설명해 주세요.', additional_kwargs={}, response_metadata={})]
<class 'langchain_core.messages.ai.AIMessage'>
## ChatGPT 모델의 학습 원리

ChatGPT는 **대규모 트랜스포머(Transformer) 기반 언어 모델**이며, 크게 **두 단계**의 학습 과정을 거칩니다.

1. **사전학습(Pre‑training)**
2. **미세조정(Fine‑tuning)·RLHF(강화학습을 통한 인간 피드백)**  

아래에서는 각각의 단계와 핵심 메커니즘을 상세히 살펴보겠습니다.

---

### 1. 사전학습 (Pre‑training)

| 단계 | 내용 | 핵심 포인트 |
|------|------|--------------|
| **데이터 수집** | 인터넷에 존재하는 방대한 텍스트(웹 페이지, 책, 논문, 대화 로그 등)를 크롤링하고, 저작권·품질을 고려해 필터링합니다. | 데이터 규모가 수백 GB ~ 수 TB 수준이며, 다양하고 풍부한 언어 현상을 포함합니다. |
| **토크나이징 (Tokenization)** | 텍스트를 **토큰**(sub‑word 단위)으로 변환합니다. 일반적으로 **Byte‑Pair Encoding (BPE)** 혹은 **SentencePiece** 같은 알고리즘을 사용합니다. | 1개의 토큰은 보통 1~4개의 문자에 해당하고, 어휘집(vocabulary) 크기는 30k~100k 정도입니다. |
| **모델 아키텍처** | **Transformer Decoder** 구조를 

In [10]:
# 체인을 생성하여 호출하기
#llm = ChatOpenAI(model="gpt-3.5-turbo-0125")

chain = chat_prompt | llm | StrOutputParser()

response = chain.invoke({"topic":"AI", "model_name":"ChatGPT"})
print(type(response))
print(response)

<class 'str'>
## ChatGPT 모델이 학습되는 과정  
(한국어로 자세히 설명합니다)

### 1. 기본 개념
- **대규모 언어 모델 (LLM)**: 수십억 개 이상의 파라미터를 가진 **Transformer** 기반 신경망.  
- **목표**: 주어진 텍스트(문맥) 다음에 올 가장 가능성이 높은 토큰(단어 혹은 서브워드)을 예측하는 것.  
- **학습 방식**: 크게 **사전 학습(Pre‑training)** → **미세 조정(Fine‑tuning)** → **강화 학습 기반 인간 피드백(RLHF)** 로 구분됩니다.

---

## 2. 사전 학습 (Pre‑training)

### 2.1 데이터 수집 & 전처리
| 단계 | 내용 |
|------|------|
| **데이터 소스** | 웹 페이지, 책, 위키피디아, 뉴스, 포럼 등 다양한 공개 텍스트. |
| **크기** | 수백 GB ~ 수 TB 규모 (예: GPT‑4는 수조 토큰). |
| **클리닝** | 저작권 침해, 개인 정보, 명백히 부적절한 내용 등을 필터링. |
| **토크나이징** | **Byte‑Pair Encoding (BPE)** 혹은 **SentencePiece** 같은 서브워드 토크나이저를 사용해 텍스트를 **토큰**(보통 1~4개의 문자)으로 변환. 토크나이저는 전체 코퍼스에서 가장 빈번한 문자/문자열 쌍을 반복적으로 병합해 vocab(어휘집)을 만든다. |

### 2.2 모델 아키텍처 – Transformer Decoder
- **입력**: 토큰 시퀀스 `x₁, x₂, …, xₙ` (각 토큰은 정수 인덱스로 변환 후 임베딩 벡터로 매핑).  
- **포지셔널 인코딩**: 순서 정보를 전달하기 위해 각 위치에 sinusoidal 혹은 학습 가능한 위치 임베딩을 더한다.  
- **다중 헤드 셀프 어텐션**:  
  \[
  \text{Attention}(Q,K,V) = \text{softmax}\!\left(\frac{QK^{\top}}{\sqrt{d_k}

#### 3) ChatPromptTemplate
* SystemMessagePromptTemplate와 HumanMessagePromptTemplate 클래스 사용
* 객체 지향적 접근 - Message 객체를 독립적으로 생성 가능
* 여러 조건에 따라 다른 시스템 메시지 선택

```python
if user_is_beginner:
    system_message = SystemMessagePromptTemplate.from_template("초보자를 위한 설명: {topic}")
else:
    system_message = SystemMessagePromptTemplate.from_template("전문가를 위한 상세 분석: {topic}")
```

In [11]:
# ChatMessagePromptTemplate 활용

from langchain_core.prompts import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
    AIMessagePromptTemplate,
    ChatMessagePromptTemplate
)
from langchain_openai import ChatOpenAI

# 개별 메시지 템플릿 정의
system_message = SystemMessagePromptTemplate.from_template(
    "You are an AI expert in {topic}. Please provide clear and detailed explanations."
)
user_message = HumanMessagePromptTemplate.from_template(
    "{question}"
)
ai_message = AIMessagePromptTemplate.from_template(
    "This is an example answer about {topic}."
)

# ChatPromptTemplate로 메시지들을 묶기
chat_prompt = ChatPromptTemplate.from_messages([
    system_message,
    user_message,
    ai_message
])

# 메시지 생성
messages = chat_prompt.format_messages(topic="AI", question="What is deep learning?")

# LLM 호출
#llm = ChatOpenAI(model="gpt-3.5-turbo-0125")
response = llm.invoke(messages)

# 결과 출력
print(response.content)

**Deep learning** is a sub‑field of machine learning (and, by extension, artificial intelligence) that focuses on algorithms called *artificial neural networks*—especially deep neural networks with many layers. These networks are inspired loosely by the structure and function of the human brain, though they are far simpler and mathematically defined.

Below is a step‑by‑step overview of what deep learning is, how it works, and why it matters.

---

## 1. The Big Picture

| Concept | Relationship |
|---------|--------------|
| **Artificial Intelligence (AI)** | The broad goal of creating machines that can perform tasks that normally require human intelligence (reasoning, perception, language, etc.). |
| **Machine Learning (ML)** | A subset of AI that gives computers the ability to learn from data rather than being explicitly programmed. |
| **Deep Learning (DL)** | A subset of ML that uses deep (many‑layered) neural networks to automatically learn hierarchical representations of data. |

#### ChatMessagePromptTemplate는 여러 종류의 메시지(시스템, 인간, AI)를 조합하여 복잡한 프롬프트를 생성할 때 유용합니다.
* SystemMessagePromptTemplate: 이 템플릿은 AI 모델에게 역할을 부여하거나 전반적인 규칙을 설정하는 시스템 메시지를 만듭니다. 위의 예시에서는 "번역을 도와주는 유용한 도우미"라는 역할을 지정합니다.
* HumanMessagePromptTemplate: 이 템플릿은 사용자의 질문이나 요청을 담는 인간 메시지를 만듭니다. 아래의 예시에서는 번역할 텍스트를 입력받습니다.
* ChatPromptTemplate.from_messages: 이 클래스 메서드는 시스템 메시지, 인간 메시지 등 여러 종류의 MessagePromptTemplate 객체들을 리스트로 받아 하나의 채팅 프롬프트 템플릿으로 통합합니다.
* format_messages: 이 메서드는 정의된 템플릿에 실제 값을 채워 넣어 [SystemMessage, HumanMessage] 형태의 리스트를 반환합니다. 이 리스트는 채팅 모델(Chat Model) 에 바로 전달될 수 있습니다.

In [12]:
# 필요한 라이브러리 임포트
from langchain.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate

# 1. SystemMessagePromptTemplate와 HumanMessagePromptTemplate 생성
# SystemMessagePromptTemplate는 모델의 페르소나 또는 기본 지침을 설정합니다.
system_template = "You are a helpful assistant that translates {input_language} to {output_language}."
system_message_prompt = SystemMessagePromptTemplate.from_template(system_template)

# HumanMessagePromptTemplate는 사용자로부터 받는 입력 프롬프트를 정의합니다.
human_template = "{text_to_translate}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

# 2. ChatPromptTemplate 생성
# 위에서 만든 두 템플릿을 리스트로 묶어 ChatPromptTemplate을 만듭니다.
chat_prompt_template = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])

# 3. 프롬프트 포맷팅
# chat_prompt_template.format_messages()를 사용하여 최종 메시지 리스트를 생성합니다.
# 이 함수는 딕셔너리 형태의 입력 변수를 받습니다.
formatted_prompt = chat_prompt_template.format_messages(
    input_language="English",
    output_language="Korean",
    text_to_translate="I love programming."
)

# 4. 결과 출력
print(formatted_prompt)

# LLM 호출
response = llm.invoke(formatted_prompt)

# 결과 출력
print(response.content)

[SystemMessage(content='You are a helpful assistant that translates English to Korean.', additional_kwargs={}, response_metadata={}), HumanMessage(content='I love programming.', additional_kwargs={}, response_metadata={})]
나는 프로그래밍을 사랑해요.


##### 4) FewShotPromptTemplate
* FewShotPromptTemplate은 모델이 특정 형식을 따르게 하거나, 일관된 응답을 생성하도록 유도할 때 유용합니다.
* 도메인 지식이 필요하거나, AI가 오답을 줄이고 더 신뢰할 만한 답변을 생성하도록 해야 할 때 효과적입니다.

##### 4-1) PromptTemplate을 사용하지 않는 경우

In [13]:
# PromptTemplate을 사용하지 않는 경우
from langchain_openai import ChatOpenAI

# model
#llm = ChatOpenAI(model="gpt-3.5-turbo")

# chain 실행
result = llm.invoke("태양계의 행성들을 간략히 정리해 주세요.")

print(type(result))
print(result.content)

<class 'langchain_core.messages.ai.AIMessage'>
## 태양계 행성 개요 (간략 정리)

| 순서 | 행성 | 종류 | 평균 거리(태양으로부터) | 주요 특징 |
|------|------|------|------------------------|-----------|
| 1 | **수성 (Mercury)** | 암석행성 | 0.39 AU (약 5.8 백만 km) | 가장 작은·가장 뜨거운(낮)·극심한 온도 차 |
| 2 | **금성 (Venus)** | 암석행성 | 0.72 AU (약 1.08 억 km) | 두꺼운 이산화탄소 대기·극강 온실효과·자전이 역행 |
| 3 | **지구 (Earth)** | 암석행성 | 1 AU (약 1.5 억 km) | 물·대기·생명 존재, 자기장이 강함 |
| 4 | **화성 (Mars)** | 암석행성 | 1.52 AU (약 2.28 억 km) | 얇은 이산화탄소 대기·극지에 물 얼음·과거 물 흐름 흔적 |
| 5 | **목성 (Jupiter)** | 가스행성 | 5.20 AU (약 7.78 억 km) | 가장 큰 행성·강력한 자기장·대적점(거대한 폭풍)·다수의 위성 |
| 6 | **토성 (Saturn)** | 가스행성 | 9.58 AU (약 1.43 억 km) | 눈에 띄는 고리 시스템·다수의 위성(특히 타이탄) |
| 7 | **천왕성 (Uranus)** | 얼음거대행성 | 19.2 AU (약 2.87 억 km) | 자전축이 98° 기울어져 ‘옆으로 누운’ 형태·청청색 대기 |
| 8 | **해왕성 (Neptune)** | 얼음거대행성 | 30.1 AU (약 4.5 억 km) | 가장 바람이 강한 행성·청청색 대기·트라이톤 위성(역행 궤도) |

### 주요 포인트
- **암석행성**(수성·금성·지구·화성)은 고체 표면을 가지고 있으며, 태양에 가까울수록 크기가 작다.  
- **가스행성**(목성·토성)은 주로 수소·헬륨으로 이루어져 있고, 거대한 대기와 강력한 자기장을 가진다.  
- 

##### 4-2) FewShotChatMessagePromptTemplate 사용하는 경우

In [18]:
# FewShotChatMessagePromptTemplate 사용하는 경우
from langchain_core.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate
from langchain_openai import ChatOpenAI

examples = [
    {
        "input": "뉴턴의 운동 법칙을 요약해 주세요.",
        "output": """### 뉴턴의 운동 법칙
1. **관성의 법칙**: 힘이 작용하지 않으면 물체는 계속 같은 상태를 유지합니다.
2. **가속도의 법칙**: 물체에 힘이 작용하면, 힘과 질량에 따라 가속도가 결정됩니다.
3. **작용-반작용 법칙**: 모든 힘에는 크기가 같고 방향이 반대인 힘이 작용합니다."""
    },
    {
        "input": "지구의 대기 구성 요소를 알려주세요.",
        "output": """### 지구 대기의 구성
- **질소 (78%)**: 대기의 대부분을 차지합니다.
- **산소 (21%)**: 생명체가 호흡하는 데 필요합니다.
- **아르곤 (0.93%)**: 반응성이 낮은 기체입니다.
- **이산화탄소 (0.04%)**: 광합성 및 온실 효과에 중요한 역할을 합니다."""
    }
]

# 예제 프롬프트 템플릿
example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}"),
        ("ai", "{output}"),
    ]
)

# FewShotChatMessagePromptTemplate 적용
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

# 최종 프롬프트 구성
final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "당신은 초등학생도 쉽게 이해할 수 있도록 쉽게 설명하는 과학 교육자입니다."),
        few_shot_prompt,
        ("human", "{input}"),
    ]
)

# 모델 생성 및 체인 구성
#llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.0)
chain = final_prompt | llm
print(chain)

# 테스트 실행
result = chain.invoke({"input": "태양계의 행성들을 간략히 정리해 주세요."})
print(result.content)

first=ChatPromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='당신은 초등학생도 쉽게 이해할 수 있도록 쉽게 설명하는 과학 교육자입니다.'), additional_kwargs={}), FewShotChatMessagePromptTemplate(examples=[{'input': '뉴턴의 운동 법칙을 요약해 주세요.', 'output': '### 뉴턴의 운동 법칙\n1. **관성의 법칙**: 힘이 작용하지 않으면 물체는 계속 같은 상태를 유지합니다.\n2. **가속도의 법칙**: 물체에 힘이 작용하면, 힘과 질량에 따라 가속도가 결정됩니다.\n3. **작용-반작용 법칙**: 모든 힘에는 크기가 같고 방향이 반대인 힘이 작용합니다.'}, {'input': '지구의 대기 구성 요소를 알려주세요.', 'output': '### 지구 대기의 구성\n- **질소 (78%)**: 대기의 대부분을 차지합니다.\n- **산소 (21%)**: 생명체가 호흡하는 데 필요합니다.\n- **아르곤 (0.93%)**: 반응성이 낮은 기체입니다.\n- **이산화탄소 (0.04%)**: 광합성 및 온실 효과에 중요한 역할을 합니다.'}], input_variables=[], input_types={}, partial_variables={}, example_prompt=ChatPromptTemplate(input_variables=['input', 'output'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=[

#### 5-1) PartialPrompt 
* 프롬프트를 더 동적으로 활용할 수 있으며, AI 응답을 더 일관성 있게 조정 가능함

In [19]:
from datetime import datetime
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

# 계절을 결정하는 함수 (남반구/북반구 고려)
def get_current_season(hemisphere="north"):
    month = datetime.now().month

    if hemisphere == "north":  # 북반구 (기본값)
        if 3 <= month <= 5:
            return "봄"
        elif 6 <= month <= 8:
            return "여름"
        elif 9 <= month <= 11:
            return "가을"
        else:
            return "겨울"
    else:  # 남반구 (계절 반대)
        if 3 <= month <= 5:
            return "가을"
        elif 6 <= month <= 8:
            return "겨울"
        elif 9 <= month <= 11:
            return "봄"
        else:
            return "여름"

# 프롬프트 템플릿 정의 (부분 변수 적용)
prompt = PromptTemplate(
    template="{season}에 일어나는 대표적인 지구과학 현상은 {phenomenon}이 맞나요? {season}에 주로 발생하는 지구과학 현상을 3개 알려주세요",
    input_variables=["phenomenon"],  # 사용자 입력 필요
    partial_variables={"season": get_current_season()}  # 동적으로 계절 값 할당
)

# OpenAI 모델 초기화
#llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.0)

# 특정 계절의 현상 질의
query = prompt.format(phenomenon="태풍 발생")
result = llm.invoke(query)


# 결과 출력
print(f" 프롬프트: {query}")
print(f" 모델 응답: {result.content}")

 프롬프트: 가을에 일어나는 대표적인 지구과학 현상은 태풍 발생이 맞나요? 가을에 주로 발생하는 지구과학 현상을 3개 알려주세요
 모델 응답: 가을에 주로 발생하는 지구과학 현상 3개는 다음과 같습니다.

1.  **태풍 발생**: 태풍은 7월부터 10월까지 발생하며, 가을에 해당하는 9월과 10월에 태풍이 많이 발생합니다.
2.  **성층권 오존층의 회복**: 오존층은 자외선에 의해 파괴되며, 성층권에서 오존 분자가 자외선을 흡수하여 지구 표면으로의 자외선 도달을 막는 역할을 합니다. 
3.  **일교차 증가**: 가을이 되면서 낮과 밤의 길이가 짧아지고, 기온 차이가 커지면서 일교차가 증가합니다. 이는 지구가 자전하면서 태양과의 거리가 멀어지기 때문입니다. 일교차는 낮과 밤의 기온 차이를 의미하며, 지구의 자전과 태양과의 거리 변화로 인해 발생합니다.


In [16]:
from datetime import datetime
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

# 계절을 결정하는 함수 (남반구/북반구 고려)
def get_current_season(hemisphere="north"):
    month = datetime.now().month

    if hemisphere == "north":  # 북반구 (기본값)
        if 3 <= month <= 5:
            return "봄"
        elif 6 <= month <= 8:
            return "여름"
        elif 9 <= month <= 11:
            return "가을"
        else:
            return "겨울"
    else:  # 남반구 (계절 반대)
        if 3 <= month <= 5:
            return "가을"
        elif 6 <= month <= 8:
            return "겨울"
        elif 9 <= month <= 11:
            return "봄"
        else:
            return "여름"

# Step 1: 현재 계절 결정
season = get_current_season("north")  # 계절 값 얻기
print(f"현재 계절: {season}")

# Step 2: 해당 계절의 자연 현상 추천
prompt2 = ChatPromptTemplate.from_template(
    "{season}에 주로 발생하는 대표적인 지구과학 현상 3가지를 알려주세요. "
    "각 현상에 대해 간단한 설명을 포함해주세요."
)

# OpenAI 모델 사용
#llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.0)
llm = ChatOpenAI(
    #api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",  # Spring AI와 동일한 모델
    temperature=0.0
)

# 체인 2: 자연 현상 추천 (입력: 계절 → 출력: 자연 현상 목록)
chain2 = (
    {"season": lambda x : season}  # chain1의 출력을 season 변수로 전달
    | prompt2
    | llm
    | StrOutputParser()
)

# 실행: 현재 계절에 따른 자연 현상 추천
response = chain2.invoke({})
print(f"\n {season}에 발생하는 자연 현상:\n{response}")

현재 계절: 가을

 가을에 발생하는 자연 현상:
가을에 주로 발생하는 대표적인 지구과학 현상 3가지는 다음과 같습니다.

1.  **북극광(Aurora Borealis)**: 북극광은 북극 지역에서 발생하는 아름다운 빛의 현상입니다. 태양풍이 지구 자기장에 부딪혀 발생하며, 주로 가을과 겨울에 북극 지역에서 볼 수 있습니다. 북극광은 녹색, 빨간색, 보라색 등 다양한 색상으로 나타납니다.

2.  **블루문(Blue Moon)**: 블루문은 달이 한 달에 두 번 뜨는 현상으로, 달의 주기와 지구의 자전 주기가 맞지 않아 발생합니다. 블루문은 가을에 자주 발생하며, 달이 더 크고 밝게 보입니다.

3.  **유성우(Meteor Shower)**: 유성우는 밤하늘에 별똥이 떨어지는 현상으로, 지구가 혜성이나 소행성의 궤도를 통과할 때 발생합니다. 가을에는 페르세우스 유성우, 오리온 유성우 등 다양한 유성우가 발생합니다. 유성우는 밤하늘을 아름답게 장식하며, 운이 좋으면 많은 별똥을 볼 수 있습니다.


#### 5-2) PartialPrompt 
* API 호출 데이터, 시간 정보, 사용자 정보 등을 반영할 때 매우 유용함

In [30]:
import requests
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

# 실시간 환율을 가져오는 함수
def get_exchange_rate():
    response = requests.get("https://api.exchangerate-api.com/v4/latest/USD")
    data = response.json()
    return f"1달러 = {data['rates']['KRW']}원"

# Partial Prompt 활용
prompt = PromptTemplate(
    template="현재 {info} 기준으로 환율 정보를 알려드립니다. 이에 대한 분석을 제공해 주세요.",
    input_variables=[],  # 사용자 입력 없음
    partial_variables={"info": get_exchange_rate()}  # API에서 가져온 데이터 자동 반영
)

# LLM 모델 설정
#llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.0)

# 모델에 프롬프트 전달 및 응답 받기
response = llm.invoke(prompt.format())

# 결과 출력
print(" 프롬프트:", prompt.format())
print(" 모델 응답:", response.content)

 프롬프트: 현재 1달러 = 1377.98원 기준으로 환율 정보를 알려드립니다. 이에 대한 분석을 제공해 주세요.
 모델 응답: ## 현행 환율: 1달러 = 1377.98원

### **주요 분석 포인트**

*   **원달러 환율 상승**: 1377.98원이라는 수치는 최근 몇 년간 꾸준히 상승한 결과입니다. 
*   **글로벌 경제 요인**: 미국 연방준비제도(Fed)의 금리 인상 사이클, 글로벌 경기침체 우려, 주요 수출국의 경기 부진 등이 원달러 환율 상승에 영향을 미치고 있습니다. 
*   **국내 경제 요인**: 한국의 무역수지 적자, 외국인 투자 자금 유출, 가계 부채 증가 등도 원화 약세 요인으로 작용하고 있습니다.

### **환율 전망**

*   **상승 지속 가능성**: 미국의 강력한 경기 지표와 고금리 장기화 가능성으로 인해 당분간 원달러 환율이 1350원대를 돌파할 가능성이 있습니다.
*   **하락 가능성**: 한국의 수출 증가세 회복, 외국인 투자 자금 유입 확대, 글로벌 경기 회복 시 원달러 환율이 하락할 가능성도 있습니다.

### **투자 및 생활에 미치는 영향**

*   **수출 기업**: 원달러 환율 상승은 수출 기업의 수익성 개선에 긍정적 영향을 미칠 수 있습니다. 
*   **수입 기업 및 소비자**: 수입 기업과 소비자는 원화 약세로 인해 수입 비용 증가, 물가 상승 등에 부정적인 영향을 받을 수 있습니다.

### **대응 전략**

*   **기업**: 환율 변동 위험 관리, 수출입 계약 조건 검토, 환헤지 전략 수립 등 필요합니다. 
*   **개인**: 환율 변동에 따른 투자 전략 조정, 외화 자산 포트폴리오 관리, 환율 변동 위험에 대비한 자산 배분 전략 세우는 것이 좋습니다.
